In [1]:
import numpy as np
import gurobipy as gbp
import time
np.random.seed(352)


In [2]:
n_com=2
n_sup=2
n_dem=3

In [3]:
#Supply Demand of all commodities
Si=np.array([[[100,200],[200,700]],
             [[300,100],[500,200]],
             [[600,1200],[500,1000]]])
Dj=np.array([[[75,190],[150,320],[200,225]],
             [[120,150],[100,400],[300,250]],
             [[300,250],[400,900],[600,1500]]])
Si.shape,Dj.shape #days X nodes X commodities

((3, 2, 2), (3, 3, 2))

In [4]:
#Vehicle capacity, number of vehicles, cost of transportation from supply nodes to demand nodes
V_cap=150
V_num=np.array([5,7])
V_cost=np.array([[10,15,20],[18,13,20]])

In [14]:
#People Variables
n_pep=np.array([[25,50,30],[25,50,30]]) #no of people on each demand node,
mu=[4,10] # consumption rate (comm req per day)

In [20]:
n_pep[1][2]

30

In [6]:
n_days=len(Si)
days_range=range(n_days)
days_range

range(0, 3)

In [7]:
# Indices & Variable Names
supply_nodes = n_sup
demand_nodes = n_dem
supply_nodes_range = range(n_sup)
demand_nodes_range = range(n_dem)
comm_range=range(n_com)
all_nodes_len = n_sup*n_dem
ALL_nodes_range = range(all_nodes_len)

print (supply_nodes_range, demand_nodes_range,comm_range, all_nodes_len)

range(0, 2) range(0, 3) range(0, 2) 6


In [8]:
Pc=np.array([Dj[ix]/Dj[ix].sum(axis=0) for ix in range(len(Dj))])
Pc

array([[[0.17647059, 0.2585034 ],
        [0.35294118, 0.43537415],
        [0.47058824, 0.30612245]],

       [[0.23076923, 0.1875    ],
        [0.19230769, 0.5       ],
        [0.57692308, 0.3125    ]],

       [[0.23076923, 0.09433962],
        [0.30769231, 0.33962264],
        [0.46153846, 0.56603774]]])

In [43]:
unserved_people=[[0,0,0]]
prev_inventory=[Si[0]]
unserved_people,prev_inventory

([[0, 0, 0]], [array([[100, 200],
         [200, 700]])])

In [44]:
prev_inventory
# decision_var[1][0][0]

[array([[100, 200],
        [200, 700]])]

In [31]:
day,orig,comm

(1, 0, 0)

In [47]:
for day in days_range:
    print ('#'*50,'  Day',day,'  ','#'*50)
    # Create Model, Set MIP Focus, Add Variables, & Update Model
    m = gbp.Model(' -- The Multi Commodity Vehicle Transportation Problem -- ')

    # Set MIP Focus to 2 for optimality
    m.setParam('MIPFocus', 2)
    # m.setParam(gbp.GRB.Param.PoolSearchMode, 1)
    # m.setParam(gbp.GRB.Param.PoolGap, 0.10)

    decision_var = []
    vehicles_var=[]
    unserved_var=[]
    for orig in supply_nodes_range:
        decision_var.append([])
        vehicles_var.append([])
        for dest in demand_nodes_range:
            decision_var[orig].append([])
            vehicles_var[orig].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                              name='S'+str(orig+1)+'_D'+str(dest+1)+'_V'))
            for comm in comm_range:
    #             print (comm,decision_var)
                decision_var[orig][dest].append(m.addVar(vtype=gbp.GRB.INTEGER, 
    #                                         obj=Cij[orig][dest],
    #                                            obj=1,
                                            name='S'+str(orig+1)+'_D'+str(dest+1)+'_c'+str(comm+1)))
    for dest in demand_nodes_range:
        unserved_var.append([])
        for comm in comm_range:
            unserved_var[dest].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                              name='D'+str(dest+1)+'_c'+str(comm+1)+'_U'))
    # Update Model Variables
    m.update()       


    #objective function
    m.setObjective(
        gbp.quicksum(gbp.quicksum((int(Dj[day][dest][comm])-gbp.quicksum(decision_var[orig][dest][comm] for orig in supply_nodes_range))*(Pc[day][dest][comm])
                                for dest in demand_nodes_range) for comm in comm_range)+
        gbp.quicksum(gbp.quicksum(vehicles_var[orig][dest]*V_cost[orig][dest] for dest in demand_nodes_range) for orig in supply_nodes_range)+
        gbp.quicksum(gbp.quicksum(unserved_var[dest][comm] for comm in comm_range) for dest in demand_nodes_range),
                            gbp.GRB.MINIMIZE)

    m.update()

    # Add Supply Constraints
    for orig in supply_nodes_range:
        for comm in comm_range:
            m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                     for dest in demand_nodes_range) - Si[day][orig][comm] - prev_inventory[day][orig][comm] <= 0)
    # Add Demand Constraints
    for dest in demand_nodes_range:  
        for comm in comm_range:
            m.addConstr(gbp.quicksum(decision_var[orig][dest][comm] 
                                     for orig in supply_nodes_range) - Dj[day][dest][comm] <= 0)
    #Add vehicle constraints
    for orig in supply_nodes_range:
        m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                 for dest in demand_nodes_range for comm in comm_range) - V_cap*V_num[orig] <=0)
    for orig in supply_nodes_range:
        m.addConstr(gbp.quicksum(vehicles_var[orig][dest] for dest in demand_nodes_range) - V_num[orig] <=0)

    for orig in supply_nodes_range:
        for dest in demand_nodes_range:
            m.addConstr(-sum(decision_var[orig][dest][comm]
                                for comm in comm_range)/V_cap + vehicles_var[orig][dest]>=0)
    for orig in supply_nodes_range:
        for dest in demand_nodes_range:
            m.addConstr(-sum(decision_var[orig][dest][comm]
                                for comm in comm_range)/V_cap + vehicles_var[orig][dest]<=1)

    #Add unserved people contstraints
    for dest in demand_nodes_range:
        for comm in comm_range:
            m.addConstr(unserved_var[dest][comm]<=n_pep[comm][dest])

    for dest in demand_nodes_range:
        for comm in comm_range:
            m.addConstr(sum(decision_var[orig][dest][comm] for orig in supply_nodes_range)-
                        ((n_pep[comm][dest]-unserved_var[dest][comm])*mu[comm])>=0)
    #  Optimize and Print( Results)
    m.optimize()
    m.write('path.lp')

#     m.display()
    prev_inventory.append([])
    for orig in supply_nodes_range:
        prev_inventory[day+1].append([])
        for comm in comm_range:
            prev_inventory[day+1][orig].append(sum(decision_var[orig][dest][comm].x
                                 for dest in demand_nodes_range))

#     print (prev_inventory)

    selected = {}
    Closed = []
    for v in m.getVars():
        var = '%s' % v.VarName
        units=int(v.x)
        selected[var] = units
    #     print (var,units)
        if (var[-1]=='V'):
            print ('-'*100)
            print( '| Day:: ',day,' Supply Facility #', var[:2], 'is sending', units, \
                  'vehicles to Demand Facility #', var[3:5]) 

        elif var[-2]=='c':
            print( '| Day:: ',day,' Supply Facility #', var[:2], 'is shipping', units, \
                                                'units of commodity',var[-2:], 'to Demand Facility #', var[3:5])
        elif var[-1]=='U':
            print ('-'*100)
            print( '| Day:: ',day,' Demand Facility #', var[:2], 'is not serving', units, \
                                                'people with commondity#', var[3:5])
        else:
            print ('Hiiiiiiii')

    print( '******************************************************************************')
    print( '    | Objective Value --------------------- ', int(m.objVal))
    print( '    | Supply Facilities ------------------- ', len(Si))
    print( '    | Total Supply Units ------------------ ', Si.sum())
    print( '    | Demand Facilites -------------------- ', len(Dj))
    print( '    | Total Demand Units ------------------ ', Dj.sum())
    print( '    | Total Potential Combinations -------- ', len(Si)*len(Dj))
    print( '    | Actual Combinations  ---------------- ', len(selected))
    # print( '    | Real Time to Optimize (sec.) -------- ', t2)
    print( '******************************************************************************')
    print( '  --  The Transportation Simplex with Gurobi --')


##################################################   Day 0    ##################################################
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 38 rows, 24 columns and 102 nonzeros
Model fingerprint: 0x200ce9b2
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 627.6070428
Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 24 columns, 93 nonzeros
Variable types: 0 continuous, 24 integer (0 binary)
Presolve removed 14 rows and 6 columns
Presolved: 17 rows, 18 columns, 51 nonzeros


Root relaxation: objective 1.670667e+02, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj 

   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 38 rows, 24 columns and 102 nonzeros
Model fingerprint: 0x082c23de
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+01]
  Objective range  [9e-02, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 1805.4092888
Presolve removed 9 rows and 0 columns
Presolve time: 0.00s
Presolved: 29 rows, 24 columns, 87 nonzeros
Variable types: 0 continuous, 24 integer (0 binary)
Presolve removed 14 rows and 6 columns
Presolved: 15 rows, 18 columns, 45 nonzeros


Root relaxation: objective 1.023540e+03, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1023.54040    0    2 1805.40929 1023.54040  43.3%     -    0s
H    0     0        